# Evolution including Virus Family hybrid Clustering based on artificially mutated K-mers

## Milestones

- [x] HDBSCAN github errors
    - need to find version without problems
    - if now finding one revert back to MA version
    - revert back to Masterthesis and update jupyter lab, git and ressource
- [x] better inclusion of R, N, ... in the kmer
    - implemented, maybe need adjustment by value
    - if frature of missing higher than threshold garbage
    - if not fill missing by possible constellations
- [x] evolution on reading frame
    - difficult with amino conservation ORF tracker necessary
        - e.g. BLOSUM etc.
    - nucleotide exchange values used now, instead of amino exchange
        - usage of Kimura's two-parameter model
        - alpha and beta of user choice
- [ ] stable parameters 
    - best would be algorithmic solution here
        - number of clusters
            - neighbors -> distance matrix -> kneedle algorithm -> epsilon
        - sample number
            - cluster number extraction algorithms -> sample 
    - alpha value (A -> G, C -> T)
    - beta value (...)
    - for more flexibility algorithmis solution was postponed to a later release

- [ ] global local hybrid clustering (GLHC)
    - idea war rejected first due to the necessary nxn space in worst case
    - maybe precalculation reinclude into clustering in the first place useful at some point
        - precalculation: nxn
        - no precalculation: nxk 
        - could be usefull in the future to change to precalculation
        - necessary for this step is the stepwise calculation of the precalc matrix
            - d(acc1, acc2), d(acc1, acc3), d(acc1, acc4), ..., d(acc1, accn)
            - d(acc2, ....) find a way to calculate the rest based on the first line
    - centroids -> single linkage(centroids)
        - difficulat to validate the quality of this method
    - all point cluster distance (apcd) instead of centroids
- [ ] k-mer implementation by taking only existing values into account
    - copy and update the old code from the project

## Implementation Blueprint

![Class2](Clusterer.svg)

## Packages

### Clustering

In [1]:
import numpy as np
import pandas as pd
import itertools as it
from Bio import SeqIO
from Bio.Seq import Seq
import math
import re
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import multiprocessing as mp
import umap
import hdbscan
import scipy.spatial.distance as ssd
from scipy.cluster import hierarchy
from tqdm import tqdm

### Alignment

In [2]:
from io import StringIO
from Bio import AlignIO
from Bio.Align.Applications import MafftCommandline
from IPython.display import display
from Bio.Phylo.Applications import RaxmlCommandline

## Classes and Definitions

### Clustering

In [3]:
class Vectors(object):
    
    def __init__(self, k = 7, identifier = 0, split = None, quality = {'':0}, variable = 2, state = 1.0, alpha = 0.0, beta = 0.0, init = 0.0, procs = 4, preprocess = True):
    
        self.k = k
        self.quality = quality
        self.identifier = identifier
        self.split = split
        self.variable = variable
        self.nucleotides = ['A', 'C', 'G', 'T']
        self.purines = ['A', 'G']
        self.pyrines = ['C', 'T']
        self.exotics = ['R', 'Y', 'W', 'S', 'M', 'K', 'B', 'H', 'D', 'V', 'N']
        #self.substit = dict.fromkeys(map(ord, self.nucleotides), None)
        #self.exist = dict.fromkeys(map(''.join, it.product(self.nucleotides, repeat = self.k)), init)        
        self.state = state
        self.init = init
        self.procs = procs
        self.alpha = alpha
        self.beta = beta
        self.preprocess = preprocess
        self.exchange = {
            'A':['A'],
            'C':['C'],
            'G':['G'],
            'T':['T'],
            'R':['A', 'G'],
            'Y':['C', 'T'],
            'W':['A', 'T'],
            'S':['C', 'G'],
            'M':['A', 'C'],
            'K':['G', 'T'],
            'B':['G', 'C', 'T'],
            'H':['A', 'C', 'T'],
            'D':['A', 'G', 'T'],
            'V':['A', 'C', 'G'],
            'N':['A', 'C', 'G', 'T'],
        } 
        self.kimura = {}
        for nuc in self.nucleotides:
            self.kimura[nuc] = {}
            if self.alpha != 0.0: 
                a = self.purines if nuc in self.purines else self.pyrines
                pos = 0 if a.index(nuc) != 0 else 1
                self.kimura[nuc][a[pos]] = self.alpha
            if self.beta != 0.0:
                b = self.purines if nuc not in self.purines else self.pyrines
                self.kimura[nuc][b[0]] = self.beta
                self.kimura[nuc][b[1]] = self.beta
    
    def countRows(self, infile):
        
        index = []
        record = SeqIO.index(infile, "fasta", key_function = lambda entry: entry.split(self.split)[self.identifier] if type(self.identifier) == int else re.search('.*' + self.identifier + '([^|]+).*', entry)[1])
        for accession in record.keys():
            
            entry = record[accession]
            header = entry.description.split(self.split)
            sequence = str(entry.seq)
            #missing = len(sequence.translate(self.substit))
            #fracture = float(len(sequence)/missing) if missing else 0 
            missing = re.findall('['+''.join(self.exotics)+']+', sequence)
            
            try:
                if all([re.match(i, header[self.quality[i]], re.IGNORECASE) for i in self.quality]) == True and (0 if not missing else len(max(missing, key=len))) <= self.variable:
                #if all([re.match(i, header[self.quality[i]], re.IGNORECASE) for i in self.quality]) == True and fracture <= 0.9:
                    index.append(accession)
            except:
                pass
        
        return(index, record)
    
    def preprocessKmer(self, sequence):
        
        exist = []
        for i in range(len(sequence) - self.k + 1):
            kmer = sequence[i:i+self.k]
            main = list(map(''.join, it.product(*[self.exchange.get(j) for j in kmer])))

            for sub in main:
                exist.append(sub)
                #if self.kimura:
                #    for l, nuc in enumerate(sub):
                #        for mut in self.kimura[nuc].keys():
                #            mutation = sub[:l] + mut + sub[l+1:]
                            #self.exist[sub] = 0
                #            exist.append(mutation)

        return(exist)
    
    def calculateKmer(self, data):
        
        temporary, sequence = data
        
        for i in range(len(sequence) - self.k + 1):
            kmer = sequence[i:i+self.k]
            main = list(map(''.join, it.product(*[self.exchange.get(j) for j in kmer])))

            for sub in main:
                temporary[sub] += self.state/len(main)
                if self.kimura:
                    for l, nuc in enumerate(sub):
                        for mut in self.kimura[nuc].keys():
                            mutation = sub[:l] + mut + sub[l+1:]
                            if mutation in temporary:
                                temporary[mutation] += self.kimura[nuc][mut]

        vector = np.fromiter(temporary.values(), dtype = 'float32', count = len(temporary.keys()))/sum(temporary.values())
        temporary.clear()
        return(vector)
        
    def calculateFrequence(self, infile):
        
        index, record = self.countRows(infile)
        #matrix = np.empty((len(index), self.col, ),dtype = 'float32')
        
        if self.preprocess == True:
            with mp.Pool(self.procs) as pool:
                preprocessor = tqdm(pool.imap(self.preprocessKmer, map(lambda m: str(record[m].seq), index)), total = len(index), desc="Preprocessing")
                exist = dict(sorted(dict.fromkeys(it.chain.from_iterable(preprocessor), self.init).items()))

        else:
            exist = dict.fromkeys(map(''.join, it.product(self.nucleotides, repeat = self.k)), self.init)    
                
        shared = it.repeat(exist,len(index))
        
        with mp.Pool(self.procs) as pool:
            calculator = tqdm(pool.imap(self.calculateKmer, zip(shared, map(lambda m: str(record[m].seq), index))), total = len(index), desc="Calculation")
            matrix = np.fromiter(it.chain.from_iterable(calculator), dtype = 'float32', count = len(exist.keys()) * len(index))
            matrix.shape = len(index), len(exist.keys())
            #for pos, vector in enumerate(calculator):
            #    matrix[pos] = vector
                
        return(index, matrix)

- execution can still be faster ca. 15-20min for segment 4 is still slow, lul now its 2 at max
    - inclusion of mutation increased the runtime by factor 5-10, nvm multipressing easy
    - multiprocessing difficult to implement (dicts, fast calculation of single instances high overhang)
- all mutations and all unkown kmers (including e.g. Ns) are counted with state or respective alpha beta
    - maybe split value by their number

In [4]:
def Cluster(linkage, min_clust, num_clust):

    x = 0.0
    y = 1.0
    cluster = linkage.get_clusters(cut_distance = x, min_cluster_size = min_clust)
    n = cluster.max().item()
    
    while n != num_clust:

        if n < num_clust and n != -1:
            x = x - y
            y = y * 0.1

        else:
            x = x + y

        cluster = linkage.get_clusters(cut_distance = x, min_cluster_size = min_clust)
        n = cluster.max().item()
        
        if x != 0.0 and n == -1:
            print('Error.')
            break
        
    return(cluster)

In [5]:
def workerCentroid(subtuple):
    
    subl2, i = subtuple
    indexdist = subl2.index.tolist()
    calcdist = map(lambda x: ssd.cdist([x[1]], subl2, 'euclidean').mean(), subl2.iterrows())
    framedist = pd.DataFrame(np.fromiter(calcdist, dtype = 'float32', count = len(subl2)), index = indexdist)
    return(framedist.idxmin().item(), i)

In [6]:
def Centroid(framecluster, framel2):
    
    groups = framecluster.query('cluster != -1').groupby('cluster').groups
    
    with mp.Pool(procs) as pool:
        result = pool.imap(workerCentroid, map(lambda match: (framel2.loc[groups[match]], match), groups.keys()))
        centroid = pd.DataFrame(result, columns = ['accession', 'cluster']).set_index('accession')
        
    return(centroid)

In [7]:
def Linkage(framecentroid, framel2, procs):
    
    subl2 = framel2.loc[framecentroid.index.tolist()]
    distance = ssd.cdist(subl2, subl2, 'euclidean')
    linkage = hierarchy.linkage(distance, method = 'single', metric = 'euclidean')
    
    return(linkage, distance)

In [8]:
def fillGaps(frame):
    
    array = frame.unique()
    array = array[array != '']
    if len(array) == 1:
        frame.replace('', array[0], inplace = True)
        frame.fillna(array[0], inplace = True)
    else:
        frame.replace('', 'na', inplace = True)
        frame.fillna('na', inplace = True)
        #changed to mixed from NA good decision?
        
    return(frame)

In [9]:
def curateFASTA(infile, split, identifier = 0, columns = {'':1}):
    
    meta = {ident:{} for ident in columns.keys()}
    
    for entry in SeqIO.parse(infile,'fasta'):
    
        header = entry.description.split(split)

        accession = header[identifier] if type(identifier) == int else re.search('.*' + acc_ident + '([^|]+).*', entry.description)[1]
        
        for key in columns.keys():

            position = columns[key]
            try:
                meta[key][accession] = header[position] if type(position) == int else '' if re.search('.*' + position + '([^|]+).*', entry.description) == None else re.search('.*' + position + '([^|]+)|.*', entry.description)[1]    
            except:
                meta[key][accession] = ''
         
    framemeta = pd.DataFrame(meta)
    
    return(framemeta)

### Alignment

In [10]:
def change_seq(accession):
    rec = record_dict[accession]
    #name = 'cluster_' + str(framecentroid.loc[accession].item())
    name = accession
    rec.id = name
    rec.name = name
    rec.description = name
    return(rec)

- needs some kind of error correction e.g. when only 4 sequences 60 clusters are impossible

## Main Pipeline

### Clustering

In [11]:
name = 'influenza_segment_7'
k = 7
split = '|'
#quality = {'(?!^UNKNOWN_.*)':1}
quality = {'pass':8, '7':2}
identifier = 0
variable = 0
min_clust = 2
sample = 1
num_clust =  60
n_components = 50
procs = 10
state = 1.0
alpha = 0.0
beta = 0.0
init = 0.0
preprocess = False
infile = 'A.fasta'
#infile = 'degue.fasta'
columns = {'strain':1,'segment':2,'protein':3,'type':4,'subtype':5,'year':6,'host':7,'pass':8,'season':9,'country':10,'state':11}
change = {'subtype':{'HA':'[H][0-9]+', 'NA':'[N][0-9]+'}}
#feature = None#'N'
#fill = False#True

In [12]:
vectors = Vectors(k = k, identifier = identifier, split = split, quality = quality, variable = variable, state = state, alpha = alpha, beta = beta, init = init, procs = procs, preprocess = preprocess)
index, matrixl1 = vectors.calculateFrequence(infile = infile)

Calculation: 100%|██████████████████████▉| 54557/54558 [03:06<00:00, 292.21it/s]


In [13]:
pca = PCA(n_components = n_components)
matrixpca = pca.fit_transform(matrixl1)
variance = pca.explained_variance_ratio_.sum()

In [14]:
matrixl2 = normalize(matrixpca, norm='l2')
#precomputed = ssd.cdist(matrixl2, matrixl2, 'euclidean')

In [15]:
hdbinit = hdbscan.HDBSCAN(min_samples = sample, min_cluster_size = min_clust, gen_min_span_tree = False, metric = 'euclidean', core_dist_n_jobs = -1).fit(matrixl2)
#hdbinit = hdbscan.HDBSCAN(min_samples = sample, min_cluster_size = min_clust, gen_min_span_tree = True, metric = 'precomputed').fit(precomputed)

In [16]:
linklocal = hdbinit.single_linkage_tree_
framelocal = linklocal.to_pandas().drop(['parent'], axis=1)#.set_index('parent', inplace = False)

In [17]:
cluster = Cluster(linklocal, min_clust, num_clust) 
framecluster = pd.DataFrame(cluster, columns = ['cluster'], index = index)
framecluster.index.rename('accession', inplace=True)

In [ ]:
framel2 = pd.DataFrame(matrixl2, index = index)
framecentroid = Centroid(framecluster, framel2)

In [ ]:
unclustered = len(framecluster.query('cluster == -1'))
clustered = len(index)

In [ ]:
#print(f'Vector Dimensionality:\t{matrixl1.shape[1]}\nUnclustered Fracture:\t{unclustered/clustered*100:.4f}%\nExplained Variance:\t{variance*100:.4f}%')

In [ ]:
framemeta = curateFASTA(infile = infile, split = split, identifier = identifier, columns = columns)
#treequery = framecluster.join(framemeta)
framemeta.index.rename('accession', inplace=True)

In [ ]:
if change:
    for key in change.keys():
        for col in change[key].keys():
            framemeta[col] = framemeta[key].apply(lambda x: re.search(change[key][col], x).group(0) if re.search(change[key][col], x) else 'mixed').tolist()
#if fill == True:
#    treequery[feature] = treequery.groupby(by = ['cluster'])[feature].apply(lambda x : fillGaps(x))

In [ ]:
framemeta.to_csv('meta_' + name + '.csv', index=True, header=True, sep=',', mode='w')

In [ ]:
framecluster.to_csv('cluster_' + name + '.csv', index=True, header=True, sep=',', mode='w')

In [ ]:
#treequery.replace('', 'na', inplace = True)
#treequery.replace(np.nan, 'na', inplace = True)
#treequery.to_csv('cluster_' + name + '.csv', index=True, header=True, sep=',', mode='w')

In [ ]:
count = framecluster.query('cluster != -1')['cluster'].value_counts().rename('size')
treecentroid = framecentroid.reset_index().set_index('cluster').join(count).reset_index().set_index('accession')#.join(treequery.drop('cluster', axis=1))
treecentroid.to_csv('centroids_' + name + '.csv', index=True, header=True, sep=',', mode='w')

In [ ]:
pcashow = PCA(n_components = 200)
pcashow.fit_transform(matrixl1)
exp_var_cumul = np.cumsum(pcashow.explained_variance_ratio_)
x = range(1, exp_var_cumul.shape[0] + 1)

In [ ]:
framepca = pd.DataFrame({'components':x, 'variance':exp_var_cumul})
framepca.set_index('components', inplace = True)
framepca.to_csv('pca_' + name + '.csv', index=True, header=True, sep=',', mode='w')

In [ ]:
pca3d = PCA(n_components = 3)
matrixpca3d = pca3d.fit_transform(matrixl1)
matrix3d = normalize(matrixpca3d, norm='l2')
frame3d = pd.DataFrame(matrix3d, columns = ['x', 'y', 'z'], index = index)
frame3d.index.rename('accession', inplace=True)
frame3d.to_csv('vectors_' + name + '.csv', index=True, header=True, sep=',', mode='w')

In [ ]:
frameinfo = pd.DataFrame({'name': name, 'sequences': len(index), 'cluster': num_clust, 'unclustered': unclustered, 'components': n_components, 'variance': variance}, index=[0])
frameinfo.to_csv('info_' + name + '.csv', index=False, header=True, sep=',', mode='w')

### Alignment

In [ ]:
record_dict = SeqIO.index(infile, "fasta", key_function = lambda entry: entry.split('|')[0])

In [ ]:
#_ = SeqIO.write(map(lambda x: record_dict[x], centr), "temp.fasta", "fasta")
_ = SeqIO.write(map(change_seq, treecentroid.index.tolist()), 'centroids_' + name + '.fasta', 'fasta')

In [ ]:
in_file = 'centroids_' + name + '.fasta'
mafft_cline = MafftCommandline(input = in_file, thread = 16, treeout = True)

In [ ]:
stdout, stderr = mafft_cline()
align = AlignIO.read(StringIO(stdout), "fasta")
_ = AlignIO.write(align, 'centroids_' + name + '.msa', 'fasta')

In [ ]:
with open('centroids_' + name + '.fasta.tree', 'r') as read:
    #update = re.sub('\d+_(.*?)_.*?([:|\n])', r'\1\2', read.read())
    update = re.sub('\d+_(.*?)([:|\n])', r'\1\2', read.read())
with open('centroids_' + name + '.fasta.tree', 'w') as write:
    write.write(update)

In [ ]:
raxml_cline = RaxmlCommandline(
    sequences = 'centroids_' + name + '.msa', 
    model = 'GTRGAMMA', 
    name = name + '.tree', 
    rapid_bootstrap_seed = 1234, 
    threads = procs, 
    num_replicates = 100, 
    algorithm = 'a', 
    parsimony_seed = 1234
)

In [ ]:
stdout, stderr = raxml_cline()

## Garbage Place